In [1]:
import sys

sys.path.append("../")

In [2]:
from lightning_ocr.models import ABINetVision

model = ABINetVision.load_from_folder("checkpoints/abinet").eval()

/home/mixaill76/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mixaill76/.local/lib/python3.10/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: '2.0.3' (you have '2.0.0'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
from PIL import Image

pillow_image = Image.open("./datasets/MNIST/item_0.jpg").convert("RGB")
pillow_image

In [4]:
model.predict(pillow_image)

['50419']

In [5]:
model.to_onnx("abinet.onnx")

/home/mixaill76/lightning_ocr/examples/../lightning_ocr/models/abinet.py:92: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!


In [6]:
from lightning_ocr.onnx.abinet import ABINetVisionONNX

mgp_str_onnx_model = ABINetVisionONNX("checkpoints/abinet", "abinet.onnx")
mgp_str_onnx_model.predict(
    [Image.open(f"./datasets/MNIST/item_{i}.jpg") for i in range(3)], True
)

(['50419', '21314', '353617'], [0.99736476, 0.99996316, 0.99990606])